In [ ]:
!pip install --upgrade openfe

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split, StratifiedKFold
import numpy as np
import pandas as pd

train = pd.read_parquet("PATH TO train.parquet")
test = pd.read_parquet("PATH TO test.parquet")

columns_to_drop = ['id', 'smpl'] + ['feature_3', 'feature_5', 'feature_10', 'feature_14', 'feature_15',
       'feature_17', 'feature_20', 'feature_22', 'feature_23',
       'feature_26', 'feature_36', 'feature_43', 'feature_46',
       'feature_48', 'feature_49', 'feature_54', 'feature_55',
       'feature_70', 'feature_77', 'feature_89', 'feature_92',
       'feature_98', 'feature_101', 'feature_115', 'feature_125',
       'feature_126', 'feature_130', 'feature_132', 'feature_133',
       'feature_135', 'feature_149', 'feature_151', 'feature_153',
       'feature_155', 'feature_156', 'feature_157', 'feature_181',
       'feature_187', 'feature_203', 'feature_206', 'feature_207',
       'feature_210', 'feature_231', 'feature_240', 'feature_242',
       'feature_244', 'feature_248', 'feature_252', 'feature_255',
       'feature_270', 'feature_287', 'feature_292', 'feature_308',
       'feature_330', 'feature_341', 'feature_348', 'feature_350',
       'feature_352', 'feature_368', 'feature_372', 'feature_381',
       'feature_396', 'feature_404', 'feature_405', 'feature_406',
       'feature_410', 'feature_411', 'feature_415'] + ['feature_4',
       'feature_12', 'feature_25', 'feature_57',
       'feature_60', 'feature_66', 'feature_72', 'feature_82',
       'feature_91', 'feature_102', 'feature_105', 'feature_137',
       'feature_142', 'feature_165', 'feature_176', 'feature_179',
       'feature_197', 'feature_198', 'feature_200', 'feature_246',
       'feature_249', 'feature_254', 'feature_262', 'feature_276',
       'feature_289', 'feature_299', 'feature_305', 'feature_306',
       'feature_312', 'feature_322', 'feature_337', 'feature_342',
       'feature_347', 'feature_367', 'feature_393'] + ['feature_1',
    'feature_6', 'feature_8', 'feature_9', 'feature_16', 'feature_19',
    'feature_29', 'feature_35', 'feature_37', 'feature_38', 'feature_40', 'feature_44', 'feature_47', 'feature_56', 'feature_59', 'feature_63', 'feature_65', 'feature_71', 'feature_73', 'feature_76', 'feature_79', 'feature_80', 'feature_83', 'feature_84', 'feature_85', 'feature_96', 'feature_107', 'feature_112', 'feature_114', 'feature_118', 'feature_123', 'feature_128', 'feature_129', 'feature_138', 'feature_139', 'feature_141', 'feature_143', 'feature_148', 'feature_152', 'feature_154', 'feature_158', 'feature_160', 'feature_161', 'feature_162', 'feature_169', 'feature_172', 'feature_173', 'feature_178', 'feature_182', 'feature_183', 'feature_185', 'feature_186', 'feature_191', 'feature_202', 'feature_204', 'feature_205', 'feature_211', 'feature_213', 'feature_214', 'feature_216', 'feature_217', 'feature_218', 'feature_223', 'feature_225', 'feature_226', 'feature_229', 'feature_232', 'feature_234', 'feature_236', 'feature_238', 'feature_243', 'feature_250', 'feature_257', 'feature_259', 'feature_261', 'feature_264', 'feature_265', 'feature_266', 'feature_268', 'feature_272', 'feature_273', 'feature_275', 'feature_277', 'feature_280', 'feature_286', 'feature_293', 'feature_297', 'feature_298', 'feature_301', 'feature_303', 'feature_304', 'feature_309', 'feature_311', 'feature_314', 'feature_316', 'feature_317', 'feature_318', 'feature_323', 'feature_332', 'feature_334', 'feature_335', 'feature_351', 'feature_353', 'feature_355', 'feature_359', 'feature_361', 'feature_363', 'feature_365', 'feature_366', 'feature_369', 'feature_373', 'feature_376', 'feature_377', 'feature_380', 'feature_382', 'feature_385', 'feature_389', 'feature_391', 'feature_392', 'feature_394', 'feature_398', 'feature_399', 'feature_401', 'feature_407', 'feature_408', 'feature_413', 'feature_414', 'feature_417']

train = train.drop(columns=columns_to_drop)
test = test.drop(columns=columns_to_drop)

# train.drop(columns=useless_features, inplace=True)

X = train.drop(columns=['target'], axis=1)
# X_test = test.drop(columns=['target'], axis=1)
y = train['target']


In [ ]:
# Определение категориальных признаков
categorical_columns = [col for col in X.columns if X[col].nunique() < 30]

# Преобразование категориальных признаков в числовой формат
for col in categorical_columns:
    X[col] = X[col].astype('category').cat.codes

# Пересчет размеров категорий
categorical_dims = {col: X[col].nunique() for col in categorical_columns}

# X_temp, X_test, y_temp, y_test = train_test_split(X, y, test_size=0.15, stratify=y, random_state=42)
# X_train, X_val, y_train, y_val = train_test_split(X_temp, y_temp, test_size=0.1765, stratify=y_temp, random_state=42)


In [ ]:
%%capture

from openfe import OpenFE
ofe = OpenFE()

features = ofe.fit(data=X, label=y, task='classification',
        categorical_features=categorical_columns,
        metric = 'auc', n_data_blocks=8, verbose=-1)

import pickle

with open('/content/drive/MyDrive/Gagarin hack/features_mazafuq_2.pkl', 'wb') as save:
    pickle.dump(features, save)


from openfe import transform

In [ ]:
def create_new_features(X):
    import pickle
    from openfe import transform

    with open('/content/drive/MyDrive/Gagarin hack/features_mazafuq.pkl', 'rb') as model_file:
        features = pickle.load(model_file)

    X, X = transform(X, X, features, n_jobs=4)